In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/:/scratch/wmql/tract_querier/tract_querier/nipype/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [3]:
import numpy as np

In [83]:
# gs: gold standard
# in: input
gs_cst_left_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/WMQL/Baseline_query_cst.left.vtp'
gs_cst_right_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/WMQL/Baseline_query_cst.right.vtp'

sr_cst_left_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/WMQL/WTV_query_cst.left.vtp'
sr_cst_right_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/WMQL/WTV_query_cst.right.vtp'

# implementation one / kde

In [84]:
def ComputeBhattacharyyaCoeficient(gs_bundle,sr_bundle):
    import vtk
    import numpy as np
    from scipy import stats
    ## read in each fiber bundle
    reader_gs = vtk.vtkXMLPolyDataReader()
    reader_gs.SetFileName(gs_bundle)
    reader_gs.Update()
    gs_bundle = reader_gs.GetOutput()
    #
    reader_sr = vtk.vtkXMLPolyDataReader()
    reader_sr.SetFileName(sr_bundle)
    reader_sr.Update()
    sr_bundle = reader_sr.GetOutput()
    #
    gs_pts = np.array([gs_bundle.GetPoint(i) for i in xrange(gs_bundle.GetNumberOfPoints())])
    sr_pts = np.array([sr_bundle.GetPoint(i) for i in xrange(sr_bundle.GetNumberOfPoints())])
    kde_gs = np.array([stats.gaussian_kde(gs_pts[:,i]) for i in xrange(3)])
    kde_sr = np.array([stats.gaussian_kde(sr_pts[:,i]) for i in xrange(3)])
    #
    mn = np.minimum(gs_pts.min(0), sr_pts.min(0))
    mx = np.maximum(gs_pts.max(0), sr_pts.max(0))
    bins = np.ceil((mx - mn))
    xyz = np.array([np.linspace(mn[i],mx[i],bins[i]) for i in xrange(3)])
    #
    p_gs = np.array([kde_gs[i].pdf(xyz[i]) for i in xrange(3)])
    p_sr = np.array([kde_sr[i].pdf(xyz[i]) for i in xrange(3)])
    coefs = np.array([np.sqrt(p_gs[i]*p_sr[i]).sum() for i in xrange(3)])
    print(coefs)
    return coefs.mean()

In [85]:
bc = ComputeBhattacharyyaCoeficient(gs_cst_left_fn,sr_cst_left_fn)
print(bc)

[ 0.9922218   0.99242648  0.99815557]
0.994267949673


# implementation two / histogram

In [86]:
def ComputeBhattacharyyaCoeficient(gs_bundle,sr_bundle):
    import vtk
    import numpy as np
    from scipy import stats
    ## read in each fiber bundle
    reader_gs = vtk.vtkXMLPolyDataReader()
    reader_gs.SetFileName(gs_bundle)
    reader_gs.Update()
    gs_bundle = reader_gs.GetOutput()
    #
    reader_sr = vtk.vtkXMLPolyDataReader()
    reader_sr.SetFileName(sr_bundle)
    reader_sr.Update()
    sr_bundle = reader_sr.GetOutput()
    #
    gs_pts = np.array([gs_bundle.GetPoint(i) for i in xrange(gs_bundle.GetNumberOfPoints())])
    sr_pts = np.array([sr_bundle.GetPoint(i) for i in xrange(sr_bundle.GetNumberOfPoints())])
    #
    mn = np.minimum(gs_pts.min(0), sr_pts.min(0))
    mx = np.maximum(gs_pts.max(0), sr_pts.max(0))
    bins = np.ceil((mx - mn))
    #
    gs_hist = np.array([ np.histogram(gs_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])
    sr_hist = np.array([ np.histogram(sr_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])
    #
    coefs = np.array([ np.sqrt( (gs_hist[i]*sr_hist[i])/(gs_hist[i].sum()*sr_hist[i].sum()) ).sum() for i in xrange(3) ])
    print(coefs)
    return coefs.mean()

In [87]:
bc = ComputeBhattacharyyaCoeficient(gs_cst_left_fn,sr_cst_left_fn)
print(bc)

[ 0.99936495  0.99884146  0.99933283]
0.999179746642


In [ ]:
import vtk
import numpy as np
reader_gs = vtk.vtkXMLPolyDataReader()
reader_gs.SetFileName(gs_cst_left_fn)
reader_gs.Update()
gs_bundle = reader_gs.GetOutput()

reader_sr = vtk.vtkXMLPolyDataReader()
reader_sr.SetFileName(gs_cst_left_fn)
reader_sr.Update()
sr_bundle = reader_sr.GetOutput()

In [ ]:
gs_pts = np.array([gs_bundle.GetPoint(i) for i in xrange(gs_bundle.GetNumberOfPoints())])
sr_pts = np.array([sr_bundle.GetPoint(i) for i in xrange(sr_bundle.GetNumberOfPoints())])

In [ ]:
print(gs_pts.shape)
print(sr_pts.shape)

In [ ]:
print(gs_pts.min(0))
print(sr_pts.min(0))

In [ ]:
mn = np.minimum(gs_pts.min(0), sr_pts.min(0))
mx = np.maximum(gs_pts.max(0), sr_pts.max(0))
print(mn)
print(mx)

In [ ]:
bins = np.ceil((mx - mn))
print(bins)

In [ ]:
xyz = np.array([np.linspace(mn[i],mx[i],bins[i]) for i in xrange(3)])
#print(xyz)

In [ ]:
gs_hist = np.array([ np.histogram(gs_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])
sr_hist = np.array([ np.histogram(sr_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])

In [ ]:
print(gs_hist.shape)

In [ ]:
coefs = np.array([ np.sqrt( (gs_hist[i]*sr_hist[i])/(gs_hist[i].sum()*sr_hist[i].sum()) ).sum() for i in xrange(3) ])

In [ ]:
print(coefs)

In [ ]:
coefs.mean()

In [ ]:
from scipy import stats
kde_gs = np.array([stats.gaussian_kde(gs_pts[:,i]) for i in xrange(3)])
kde_sr = np.array([stats.gaussian_kde(sr_pts[:,i]) for i in xrange(3)])

In [ ]:
print(kde_gs)

In [ ]:
print(kde_gs[0].integrate_box(mn[0]-5,mx[0]+5))

In [ ]:
p_gs = np.array([kde_gs[i].pdf(xyz[i]) for i in xrange(3)])
p_sr = np.array([kde_sr[i].pdf(xyz[i]) for i in xrange(3)])

In [ ]:
print(p_gs[0].sum())

In [ ]:
coefs = np.array([ np.sqrt(p_gs[i]*p_sr[i]).sum() for i in xrange(3) ])
print(coefs)
print(coefs.mean())

In [ ]:
print(np.sqrt(p[2]).sum())